In [16]:
import numpy as np
import pickle as pkl
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

In [17]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')  # Global average pooling

# Function to Extract Features
def extract_features(image_path):
    img = Image.open(image_path).convert('RGB')  # Resize image
    img_array = img_to_array(img)  # Convert to array
    img_array = preprocess_input(img_array)  # Preprocess for ResNet
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    features = base_model.predict(img_array)  # Extract features
    return features.flatten()  # Flatten to 1D array

In [11]:
from glob import glob

def extract_class_dir(path):
    folders = glob(path+'/*')
    features = []
    for folder in folders:
        path = folder+'/*.png'
        fns = glob(path)
        for fn in fns:
            feature = extract_features(fn)
            features.append(feature)
    return features


In [12]:
covid_features = extract_class_dir(r'dataset\New_Data_CoV2\Covid')
normal_features = extract_class_dir(r'dataset\New_Data_CoV2\Healthy')
others_features = extract_class_dir(r'dataset\New_Data_CoV2\Others')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [13]:
labels = []
features = []
for i in range(len(covid_features)):
    features.append(covid_features[i])
    labels.append('covid')
for i in range(len(normal_features)):
    features.append(normal_features[i])
    labels.append('normal')
for i in range(len(others_features)):
    features.append(others_features[i])
    labels.append('other')

In [16]:
features = np.array(features)
labels = np.array(labels)
np.save('features/3class_features.npy', features)
np.save('features/3class_labels.npy',labels)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
features = np.load(r'features\3class_features.npy')
labels = np.load(r'features\3class_labels.npy')

In [11]:
xtr,xte,ytr,yte=train_test_split(features,labels,test_size=0.2,random_state=42)

In [12]:
scaler = StandardScaler()
xs = scaler.fit_transform(xtr)
xt = scaler.transform(xte)

In [13]:
rf = RandomForestClassifier()
rf.fit(xs,ytr)
yp=rf.predict(xt)
print(accuracy_score(yte,yp))
print(confusion_matrix(yte,yp))

0.8778443113772455
[[443   1  12]
 [  8 111  27]
 [ 32  22 179]]


In [8]:
import pickle as pkl
with open('rf_modelv2.pkl','wb') as m:
    pkl.dump(rf,m)
with open('scaler.pkl', 'wb') as sc:
    pkl.dump(scaler, sc)

In [14]:
svm = SVC()
svm.fit(xs,ytr)
yp=svm.predict(xt)
print(accuracy_score(yte,yp))
print(confusion_matrix(yte,yp))

0.9341317365269461
[[448   4   4]
 [  3 128  15]
 [ 12  17 204]]


In [9]:
log=LogisticRegression()
log.fit(xs,ytr)
yp=log.predict(xt)
print(accuracy_score(yte,yp))
print(confusion_matrix(yte,yp))

0.9305389221556887
[[443   4   9]
 [  0 128  18]
 [  9  18 206]]


In [42]:
sgdc = SGDClassifier()
sgdc.fit(xs, ytr)
yp=sgdc.predict(xt)
print(accuracy_score(yte,yp))
print(confusion_matrix(yte,yp))

0.8910179640718563
[[452   2   2]
 [ 15 109  22]
 [ 38  12 183]]


In [ ]:
xt = extract_features(r'dataset\COVID_\Images\137covid_patient1_SR_2_IM00011.tif')
xs = scaler.transform([xt])
yp = svm.predict(xs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [20]:
print(yp)

['covid']


In [21]:
import pandas as pd

In [27]:
df = pd.read_csv(r'dataset\COVID_\CSV\test1.csv')
df1 = pd.read_csv(r'dataset\COVID_\CSV\train1.csv')
df = pd.concat([df,df1])
print(df.loc[df['filename']==r'137covid_patient1_SR_2_IM00011.tif', 'class'])

1    covid
Name: class, dtype: object


In [28]:
def tester(path, fn):
    xt = extract_features(path+'/'+fn)
    xs = scaler.transform([xt])
    yp = svm.predict(xs)
    yt = df.loc[df['filename']==fn, 'class']
    return [yp,yt]    

In [33]:
print(tester(r'dataset\New_Data_CoV2\Others\Patient (2)','Captura de Tela 2020-06-01 +ás 16.42.32.png'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[array(['other'], dtype='<U6'), Series([], Name: class, dtype: object)]
